In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression


In [2]:
products = []
customers = []
for i in range(5):
    product = TimeSeriesGenerator(size=1460, amountOfVariables=7)
    customer = TimeSeriesGenerator(size=1460, amountOfVariables=3)
    products.append(product)
    customers.append(customer)


In [3]:
testProduct = TimeSeriesGenerator(size= 1460, amountOfVariables=10)

In [4]:
testProduct.toDataFrame()

,Variable_0,Variable_1,Variable_2,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Response,year,week,day
2017-01-01,38.927097,47.912278,44.412233,43.202600,35.147571,36.069443,35.547501,45.008288,46.294847,39.774110,168.773607,2016,52,7
2017-01-02,38.841892,48.294326,44.390958,42.673458,36.370185,35.786788,36.250125,43.185938,46.480497,40.669259,174.040671,2017,1,1
2017-01-03,39.123898,45.793208,46.134186,44.243660,36.164154,37.372499,35.122105,43.549238,44.750885,40.127323,170.886333,2017,1,2
2017-01-04,39.009179,47.242982,44.463746,42.948387,36.639928,37.651163,34.842364,43.938287,46.503435,39.310139,176.364091,2017,1,3
2017-01-05,38.550869,46.297899,44.611455,44.286728,35.660256,37.106722,35.705426,43.658576,44.934005,40.195155,170.708670,2017,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,38.813468,45.633067,45.882332,43.006009,36.403339,37.489259,35.671834,43.932071,46.249497,40.589237,170.875604,2020,52,6
2020-12-27,39.867619,46.177575,44.952547,43.744151,35.025859,37.323770,35.893636,44.249111,44.818949,40.134748,168.685640,2020,52,7
2020-12-28,39.768344,46.686087,46.191334,44.878615,36.711071,36.037173,36.552715,43.265101,46.166204,39.570172,173.842735,2020,53,1
2020-12-29,38.192695,47.526618,44.981718,44.511194,34.817742,36.121819,35.655631,45.021009,46.769275,40.751276,173.995713,2020,53,2


In [5]:
suddenDrift(testProduct, 0, 3, magnitude=5)

In [6]:
testProduct.toDataFrame()

,Variable_0,Variable_1,Variable_2,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Response,year,week,day
2017-01-01,38.927097,47.912278,44.412233,43.202600,35.147571,36.069443,35.547501,45.008288,46.294847,39.774110,168.773607,2016,52,7
2017-01-02,38.841892,48.294326,44.390958,42.673458,36.370185,35.786788,36.250125,43.185938,46.480497,40.669259,174.040671,2017,1,1
2017-01-03,39.123898,45.793208,46.134186,44.243660,36.164154,37.372499,35.122105,43.549238,44.750885,40.127323,170.886333,2017,1,2
2017-01-04,195.045893,47.242982,44.463746,42.948387,36.639928,37.651163,34.842364,43.938287,46.503435,39.310139,311.886629,2017,1,3
2017-01-05,192.754344,46.297899,44.611455,44.286728,35.660256,37.106722,35.705426,43.658576,44.934005,40.195155,304.638985,2017,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,194.067342,45.633067,45.882332,43.006009,36.403339,37.489259,35.671834,43.932071,46.249497,40.589237,305.718221,2020,52,6
2020-12-27,199.338096,46.177575,44.952547,43.744151,35.025859,37.323770,35.893636,44.249111,44.818949,40.134748,307.190503,2020,52,7
2020-12-28,198.841720,46.686087,46.191334,44.878615,36.711071,36.037173,36.552715,43.265101,46.166204,39.570172,312.002704,2020,53,1
2020-12-29,190.963476,47.526618,44.981718,44.511194,34.817742,36.121819,35.655631,45.021009,46.769275,40.751276,306.681691,2020,53,2


In [3]:
thesis = ExperimentTracker(products, customers, scenarios())

In [4]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), make_column_selector(dtype_include=object))], remainder='passthrough')
xgb_pipe = Pipeline([
    ('onehot', onehot_cols),
('xgb', XGBRegressor())
])

In [5]:
thesis.runExperiment(MLalgo=xgb_pipe)   

Finished experiment! Elapsed time: 3.7918596267700195, total Elapsed time: 3.7918596267700195, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 3.420950174331665, total Elapsed time: 7.212809801101685, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 4.733348369598389, total Elapsed time: 11.946158170700073, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 6.39339542388916, total Elapsed time: 18.339553594589233, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished experiment! Elapsed time: 4.822110891342163, total Elapsed time: 23.161664485931396, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observe

In [6]:
results = thesis.resultsToDF()

In [7]:
results["SMAPE"] = results["SMAPE"] * 100

In [14]:
results.groupby(["Drift type", "Drift time", "Drift magnitude"]).mean()

MSE      MAPE  \
Drift type        Drift time     Drift magnitude                         
Incremental Drift Fully observed Large              8.271559  0.013419   
                                 Small              8.967114  0.013994   
                  Half observed  Large              8.840765  0.013880   
                                 Small              8.714955  0.013885   
                  Unobserved     Large            558.621137  0.031220   
                                 Small             41.975517  0.019513   
No Drift          No Drift       No Drift           8.899183  0.013788   
Sudden Drift      Fully observed Large              9.992189  0.010523   
                                 Small              8.659078  0.012509   
                  Half observed  Large             10.095365  0.010615   
                                 Small              8.750876  0.012646   
                  Unobserved     Large             10.670056  0.011019   
                                 Small              9.245467  0.012732   

                                                     SMAPE  
Drift type        Drift time     Drift magnitude            
Incremental Drift Fully observed Large            1.348734  
                                 Small            1.407519  
                  Half observed  Large            1.395954  
                                 Small            1.394941  
                  Unobserved     Large            3.517599  
                                 Small            2.001665  
No Drift          No Drift       No Drift         1.387392  
Sudden Drift      Fully observed Large            1.055851  
                                 Small            1.256672  
                  Half observed  Large            1.066106  
                                 Small            1.272128  
                  Unobserved     Large            1.108089  
                                 Small            1.281673

In [9]:
results.sort_values("SMAPE", ascending=False)

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,MAPE,SMAPE
103,3,Incremental Drift,Large,Important,Unobserved,1257.336865,0.043709,5.193841
140,6,Incremental Drift,Large,Important,Unobserved,1268.890180,0.043337,5.165307
66,0,Incremental Drift,Large,Important,Unobserved,1272.688506,0.042938,5.130331
29,None,Incremental Drift,Large,Important,Unobserved,1256.040847,0.042577,5.083662
177,"[0, 3, 6]",Incremental Drift,Large,Important,Unobserved,1289.269975,0.042280,5.078439
...,...,...,...,...,...,...,...,...
84,3,Sudden Drift,Large,Important,Half observed,8.516410,0.007762,0.777935
10,None,Sudden Drift,Large,Important,Half observed,8.340539,0.007727,0.775490
121,6,Sudden Drift,Large,Important,Half observed,8.090085,0.007608,0.762940
9,None,Sudden Drift,Large,Important,Fully observed,8.629195,0.007530,0.754743


In [10]:
results.to_csv("first_resultsAdj.csv")